# Aanmaken van de database

* dit is Jupyter Notebook `sqlite-0.ipynb` - voor het aanmaken van de database.
* voor een inleiding over het gebruik van Jupyter Notebooks: [Inleiding Jupyter Notebook](../../inleiding-jupyter.ipynb)
* de hele reeks SQLite opdrachten:
    * [SQLite 0 - init database](sqlite-0.ipynb) (om met een schone lei te beginnnen)
    * [SQLite 1 - selectie en projectie](sqlite-1.ipynb)    
    * [SQLite 2 - joins](sqlite-2.ipynb)
    * [SQLite 3 - CRUD](sqlite-3.ipynb)
    * [SQLite 4 - Schema](sqlite-4.ipynb)

## Voorbeeld
 
Bij deze opdrachten gebruiken we een voorbeeld-database met drie tabellen: `leden`, `inschrijvingen`, en `events`.
Deze database komt uit een webtoepassing; deze vind je op glitch.com. REF
Daar kun je de toepassing bekijken, uitproberen, en er een eigen versie ("remix") van maken.

## Aanmaken van de database

In de volgende opdrachten voer je allerlei queries uit op een database.
Je moeten dan eerst wel een database met inhoud hebben.
Met de onderstaande opdrachten maak je deze database.
Deze opdrachten hoef je maar één keer uit te voeren: de database blijft bestaan, met je veranderingen.
Je kunt deze opdrachten later ook uitvoeren om opnieuw te beginnen in een goed-gedefinieerde toestand.

We maken de tabel(len) aan.
We verwijderen eerst een eventueel bestaande versie van de tabel(len):
we hebben dan een goed gedefinieerde toestand.

> Opmerking: er zijn kleine verschillen in de notatie van de constraints bij het aanmaken van een tabel; MySQL gebruikt bijvoorbeeld een andere notatie dan Oracle.

### Eerste tabel: leden

De opdracht bestaat uit de volgende onderdelen:

1. het opstarten van `sqlite` (de eerste twee regels). Hierna kunnen we SQL opdrachten geven;
2. het verwijderen van de `leden`-tabel als deze bestaat (`DROP TABLE`);
3. het aanmaken van de `leden`-tabel (`CREATE TABLE`);
4. het vullen van de tabel uit een csv-bestand (dit zijn geen SQL-opdrachten);
5. een SQL-`SELECT`-opdracht om te controleren of de tabel inderdaad ingelezen is.

In [6]:
%%bash
sqlite3 example.db

DROP TABLE IF EXISTS leden;

CREATE TABLE leden(
  lidnr INTEGER PRIMARY KEY,
  voornaam VARCHAR(255) NOT NULL,
  achternaam VARCHAR(255) NOT NULL,
  email VARCHAR(255) NOT NULL UNIQUE
);

.mode csv
.import leden.csv leden

SELECT lidnr, voornaam, achternaam, email
FROM leden;

1,Hans,"de Boer",hdebo@ziggo.nl
2,Marie,Verkerk,marie@verkerk.nl
3,Jantien,Gerards,jgerards@gmail.com
4,Marie,Zandstra,marie123@ziggo.nl


We hebben een voorbeeld-inhoud van de tabel(len) in csv-bestanden.
Zo'n csv-bestand kun je gemakkelijk aanpassen in een teksteditor.
Voor het importeren van een csv-bestand gebruiken we een speciale SQLite-opdracht.

### Tweede tabel: events

De tabel `events` bevat de events waarvoor de leden kunnen inschrijven.
Elk event heeft een datum en een beschrijving.

Hiervoor volgen we hetzelfde patroon:

In [7]:
%%bash
sqlite3 example.db

DROP TABLE IF EXISTS events;

CREATE TABLE events(
  eventnr INTEGER, 
  datum VARCHAR(10) NOT NULL,
  beschrijving VARCHAR(255),
  PRIMARY KEY (eventnr),
  CONSTRAINT name UNIQUE (datum, beschrijving)
);

.mode csv
.import events.csv events

SELECT eventnr, datum, beschrijving
FROM events;

1,2019-08-21,"Schaaktoernooi Breda"
2,2019-08-28,"Schaaktoernooi Breda"
3,2019-08-28,"Schaaktoernooi Assen"
4,2019-09-15,"Schaaktoernooi Amersfoort"


### Derde tabel: inschrijvingen

Deze tabel beschrijft een N-M relatie tussen leden en inschrijvingen.
Naast de verwijzingen (via *foreign keys*) naar de andere tabellen vindt je hier de gegevens over de inschrijving (maaltijd-keuze).

In [8]:
%%bash
sqlite3 example.db

DROP TABLE IF EXISTS inschrijvingen;

CREATE TABLE inschrijvingen(
  eventnr INTEGER, 
  lidnr INTEGER,
  maaltijd VARCHAR(255),
  PRIMARY KEY (lidnr, eventnr),
  FOREIGN KEY (lidnr) REFERENCES leden (lidnr),
  FOREIGN KEY (eventnr) REFERENCES events (eventnr)
);

.mode csv
.import inschrijvingen.csv inschrijvingen

SELECT eventnr, lidnr, maaltijd
FROM inschrijvingen;

1,1,"maaltijd A"
2,1,"maaltijd B"
3,2,geen
4,3,"maaltijd A"


### Demonstratie: alle inschrijvingen

Voor een overzicht van alle inschrijvingen met de gegevens van de leden en van de events gebruiken  we een join. 
Dit is een voorproefje - in een volgend notebook werken we dit verder uit.

In [9]:
%%bash
sqlite3 example.db

SELECT evt.datum
,      evt.beschrijving
,      lid.voornaam
,      lid.achternaam
,      lid.email
,      ins.maaltijd
FROM leden lid, events evt, inschrijvingen ins
WHERE lid.lidnr=ins.lidnr AND evt.eventnr=ins.eventnr;

2019-08-21|Schaaktoernooi Breda|Hans|de Boer|hdebo@ziggo.nl|maaltijd A
2019-08-28|Schaaktoernooi Breda|Hans|de Boer|hdebo@ziggo.nl|maaltijd B
2019-08-28|Schaaktoernooi Assen|Marie|Verkerk|marie@verkerk.nl|geen
2019-09-15|Schaaktoernooi Amersfoort|Jantien|Gerards|jgerards@gmail.com|maaltijd A
